## A Dive into Earthquakes 

### Importing Data

In [ ]:
import random 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected = True)
%matplotlib inline
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

pd.set_option("display.max_columns",None)
sns.set_style("whitegrid")
plt.style.use("dark_background")

earthquake = pd.read_csv("../input/earthquake-database/database.csv")
earthquake.head(5)

In [ ]:
#Changing Date time
earthquake["Date"] = pd.to_datetime(earthquake["Date"])

### Analysis

###### Taken from https://www.usgs.gov/programs/earthquake-hazards/science-earthquakes. What is an Earthquake? An earthquake is what happens when two blocks of the earth suddenly slip past one another. The surface where they slip is called the fault or fault plane. The location below the earth’s surface where the earthquake starts is called the hypocenter, and the location directly above it on the surface of the earth is called the epicenter. 
###### Sometimes an earthquake has foreshocks. These are smaller earthquakes that happen in the same place as the larger earthquake that follows. Scientists can’t tell that an earthquake is a foreshock until the larger earthquake happens. The largest, main earthquake is called the mainshock. Mainshocks always have aftershocks that follow. These are smaller earthquakes that occur afterwards in the same place as the mainshock. Depending on the size of the mainshock, aftershocks can continue for weeks, months, and even years after the mainshock!

In [ ]:
# Data I will Be Using
data =earthquake[["Date","Time","Type","Depth","Depth Error","Magnitude","Magnitude Type","Magnitude Error","Location Source"]]
data.rename(columns = {'Magnitude Type':"Magnitude_Type","Depth Error":"Depth_Error"},inplace = True)
data.head(3)

In [ ]:
display(data.describe())
display(data.info())

In [ ]:
sns.pairplot(data,palette = "tab20",hue = "Magnitude_Type")
plt.show()

###### Types
###### We can see that there are also Nuclear Explosions, Explosions and Rock Bursts. A nuclear explosion can cause an earthquake and even an aftershock sequence. However, earthquakes induced by explosions have been much smaller than the explosion, and the aftershock sequence produces fewer and smaller aftershocks than a similar size earthquake. Not all explosions have caused earthquakes.Rock bursts, in which rocks are ejected suddenly in deep pits or tunnels, are caused by increase of stress in the surrounding rocks.

In [ ]:
display(px.pie(data,names = "Type",title = "Types",color ="Type" ,hole = .4))
display(data["Type"].value_counts())

###### (1) local magnitude (ML), commonly referred to as "Richter magnitude", (2) surface-wave magnitude (Ms), (3) body-wave magnitude (Mb), and (4) moment magnitude (Mw). Scales 1-3 have limited range and applicability and do not satisfactorily measure the size of the largest earthquakes. The moment magnitude (Mw) scale, based on the concept of seismic moment, is uniformly applicable to all sizes of earthquakes but is more difficult to compute than the other types. All magnitude scales should yield approximately the same value for any given earthquake.

In [ ]:
fig,axes = plt.subplots(1,2,figsize = (18,5))
import random
color = []
for i in range(10):
    rgb = random.uniform(0,0.3),random.uniform(0,0.5),random.uniform(0,0.5)
    color.append(rgb)

bar = sns.barplot(ax = axes[1],data = data.groupby(["Magnitude_Type"],as_index = False)["Magnitude"].mean().sort_values(by = "Magnitude"),x = "Magnitude",y ="Magnitude_Type",palette = "tab20")
pie = data["Magnitude_Type"].value_counts()[:6:].plot(ax = axes[0],kind = "pie",ylabel = " ",autopct = "%1.0f%%",colors = color,explode = [0.1,0,0,0,0,0],title = "Magnitude Type Count")
plt.show(pie)
print(data["Magnitude_Type"].value_counts())

###### Depth And Magnitude

###### Comparing Depth and Magnitude We can see certain trend. It is apparent more (recorded) earthquakes happen at shallower depths and also come in higher Magnitude whereas the deeper an earthquake is recorded, the less frequent and  powerfull they generally are.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = data.Depth,y = data.Magnitude,mode = "markers",marker_color = data.Magnitude,text = data.Type,marker = dict(showscale = True)))
fig.update_layout(title = "Depth and Magnitude",xaxis_title="Depth",yaxis_title="Magnitude")



In [ ]:
px.histogram(data,x = "Depth",y = "Magnitude",marginal="box",color = "Type")

In [ ]:
plt.figure(figsize = (15,5))
sns.kdeplot(data = data,x = "Depth",y = "Magnitude",cmap = "coolwarm",shade = True)
plt.show()

###### Looking At Location Sources:  

In [ ]:
fig,axes = plt.subplots(1,2,figsize = (20,5),dpi = 100)
l = sns.barplot(ax = axes[0],data = data.groupby("Location Source",as_index = False)["Magnitude"].count().sort_values(by = "Magnitude",ascending = False)[:10],x = "Magnitude", y = "Location Source")
sns.barplot(ax = axes[1],data = data.groupby("Location Source",as_index = False)["Magnitude"].mean().sort_values(by = "Magnitude")[:10],x = "Magnitude", y = "Location Source",palette = "tab10")
axes[0].set_title("Location With overall Most Recorded Earhquakes")
axes[1].set_title("Average Magnitude OF Locations")
l.set_xlabel("Earthquakes")
plt.show()

In [ ]:
location = data.loc[data["Location Source"].str.contains("US|ISCGEM|CI|GCMT|NC|GUC|AEIC|UNM")]
display(px.pie(location,names ="Location Source",title = "Percentage of EarthQuakes per Location",hole = .4))
data["Location Source"].value_counts()[:8]

###### Spread of Earthquakes Through the Years
###### We can see a slight rise in more intense recorded earthquakes between 1970 and 2020 however a relativley consistent amount of earthquakes below a magnitude of 7 through the years.

In [ ]:
fig = px.scatter(data,x = "Date",y = "Magnitude",color = "Type")
fig.show()
data["Magnitude"].value_counts().head(20)


In [ ]:
pip install chart_studio


###### My First attempt at a 3D plot. Basically shows the Comparison of Dpeth and Magnitude of Earthquakes throughout the years

In [ ]:
import chart_studio.plotly as py

init_notebook_mode(connected=True)
trace1 = go.Scatter3d(
    z=data.Magnitude,
    x=data.Depth,
    y=data.Date,
    mode='markers',
    marker=dict(
        size=3,
        color='rgb(120,0,0)',                    
    )
)

data1 = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0  
    )
    
)
fig = go.Figure(data=data1, layout=layout)
iplot(fig)

###### Looking At Depth And Magnitude Error

###### First of all, the depth of an earthquake is usually the most difficult part of its location to nail down with great accuracy. Since most earthquakes are deep within the crust, an error of +/- 1 or 2 km is irrelevant; in other words, it is a small error when the depth is something like 13 km. Missing Values Represent no Error. 

In [ ]:
error_ =pd.DataFrame({"Type":data[["Depth_Error","Magnitude Error"]].dtypes})
error_["Margins of Error"] = (23412-data["Depth_Error"].isnull().sum())
error_.iloc[1,1] = (23412-data["Magnitude Error"].isnull().sum())
display(error_)
print(f"Margin of Error Shows out of 23,412 data points margins of error show pieces of data that have some sort of error")

###### Errors Over The Years

In [ ]:
px.scatter(data,x = "Date",y = "Depth_Error",color = "Type")

In [ ]:
px.scatter(data,x = "Date",y = "Magnitude Error",color = "Type")

##### Feel Free to Check Out my Visualization on Tableau https://public.tableau.com/app/profile/kenan1047/viz/Earthquakes_16603753800210/Dashboard1